<a href="https://colab.research.google.com/github/ShikhaSahu/Machine_Translation/blob/master/c1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
import xlrd
loc ='/content/drive/My Drive/data_set1 - Copy.xlsx'
workbook = xlrd.open_workbook(loc)
worksheet = workbook.sheet_by_name("Sheet1")
num_rows = worksheet.nrows #Number of Rows
num_cols = worksheet.ncols #Number of Columns
f = open("text_data_file_en_hn.txt","+w")
for curr_row in range(0, num_rows, 1):
 
 for curr_col in range(0, num_cols-1, 1):
  data = worksheet.cell_value(curr_row, curr_col) # Read the data in the current cell
  f.write(data)
  if curr_col == 0:
    f.write("\t")
  else:
    f.write("\n")



f.close()

In [0]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    #table = str.maketrans('', '', string.punctuation)
    #s = [word.translate(table) for word in s]
    s = re.sub(r"[.!?]+", r" ", s)
    return s

In [0]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/text_data_file_en_hn.txt',encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [0]:
MAX_LENGTH = 22

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH #and \
        #p[0].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [27]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    for pair in range(20):
      print((pairs[pair][0].split(' ')),pairs[pair][1].split(' '))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'hin1', False)
print(random.choice(pairs))
#print(pairs)

Reading lines...
Read 2799 sentence pairs
['wow', '', ''] ['वाह', '', '']
['help', '', ''] ['बचाओ', '', '']
['jump', '', ''] ['उछलो', '', '']
['jump', '', ''] ['कूदो', '', '']
['jump', '', ''] ['छलांग', '', '']
['hello', '', ''] ['नमस्ते।']
['hello', '', ''] ['नमस्कार।']
['cheers', '', ''] ['वाह-वाह', '', '']
['cheers', '', ''] ['चियर्स', '', '']
['got', 'it', '', ''] ['समझे', 'कि', 'नहीं', '', '']
["i'm", 'ok', '', ''] ['मैं', 'ठीक', 'हूँ।']
['awesome', '', ''] ['बहुत', 'बढ़िया', '', '']
['come', 'in', '', ''] ['अंदर', 'आ', 'जाओ।']
['get', 'out', '', ''] ['बाहर', 'निकल', 'जाओ', '', '']
['go', 'away', '', ''] ['चले', 'जाओ', '', '']
['goodbye', '', ''] ['ख़ुदा', 'हाफ़िज़।']
['perfect', '', ''] ['उत्तम', '', '']
['perfect', '', ''] ['सही', '', '']
['welcome', '', ''] ['आपका', 'स्वागत', 'है।']
['welcome', '', ''] ['स्वागतम्।']


IndexError: ignored

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
